# Examen ETL: SPARK 02/02

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

Collect es una acción que toma un RDD y devuelve todos sus elementos en una lista. Una de sus principales aplicaciones es devolver el resultado de manera visible para el usuario cuando se han terminado las transformaciones. El uso más interesante de `collect` es, sin embargo, el forzar a las transformaciones a trabajar, es decir, un RDD es una serie de operaciones, no son datos per se, sin embargo, al añadir una acción al final de estas intrucciones se fuerza a que la lista de operaciones antes mencionadas sea ejecutada.

Tanto `cache` como `persist` son técnicas de optimización para cálculo de Spark (tanto iterativos como interactivos). Su principal ventajas es que almacenan resultados intermedios para su reutilización en los subsecuentes pasos. Los RDD se almacenan en la memoria cache mediante `cache`. La diferencia entre `cache` y `persist` es puramente sintáctica. `cache` es simplemente un `persist` en el que el nivel de almacenamiento viene dado por defecto a `MEMORY_ONLY`.

La aplicación de `cache` y `persist` puede resultar muy interesante en algunas ocasiones. Es requisito esencial para poder usar estas acciones que la memoria del dispositivo tenga suficiente capacidad de almacenamiento. De darse esta condición resulta útil llamar a dichas acciones cuando se va a trabajar varias veces con el mismo RDD, por ejemplo, en un bucle. Al cargarse los datos en la memoria la velocidad de este proceso será muy rápida. Otra situación en la que resulta interesante es cuando se van a realizar varias operaciones (sin necesidad de ser un bucle) sobre el mismo RDD.

Esta opción debe usarse con cautela. El proceso de almacenar los datos en memoria lleva su tiempo y en ocasiones resulta más rápido realizar la carga de los datos cada vez que cargarlos en memoria para luego reutilizarlos.

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

A la hora de transferir la información desde una base de datos estructurada hasta un data
lake se deben realizar una serie de comprobaciones que garanticen el buen traspaso de dicha
información (entendiéndose por buen traspaso que los datos sean grabados correctamente, que
no se pierdan datos importantes, que no se produzcan duplicaciones y que los datos sean
consistentes).

Para evitar estos errores, existen una serie de medidas que se pueden llevar a cabo para
reducir (o incluso eliminar) los errores. Algunas de ellas son:

- Comprobación de las dimensiones. Se miden los metadatos antes de ser transferidos y se compara con los que han sido almacenados para garantizar que todas las variables han sido traspasadas y que el número de registros traspasados si no es igual al total es al menos próximo (en ocasiones se pierden pocos registros y no es tan problemático).

- Eliminación de duplicados. Suele ser adecuado que las bases de datos lleven un índice o una identificación para cada registro. Gracias a esta buena praxis se realiza una fácil detección de duplicados o se localizan posibles registros perdidos en el traspaso.
- Comprobación de los tipos. Comprobar que los tipos de las variables no han cambiado durante el traspaso. En caso de ocurrir esto, realizar un cast al tipo adecuado.
- Parseado de datos específicos. Asegurar que los formatos especiales (variablescategóricas, fechas…) se encuentran en el formato adecuado.
- Otra medida de garantía es contrastar los datos con cargas realizadas anteriores. Si el número de datos subido en dos días diferentes presenta una gran diferencia cabe sospechar que alguno de los dos días ha habido problemas (perdida de datos o almacenamiento de datos repetidos). En este sentido se puede dibujar una gráfica con el número de datos de cada carga permitiéndose así una fácil detección de los valores atípicos. Conviene tomar como medida periodos de tiempo suficientemente amplios y uniformes, por ejemplo, meses.
- Comprobar que se mantiene la escala. Asegurarse de que no ha habido problemas con el separador decimal (punto o coma según el sistema) para ello basta inspeccionar el máximo y mínimo de cada variable. 
- Comprobar que las variables categóricas se han traspasado de manera adecuada. En ocasiones los caracteres especiales como pueden ser las ñ’s o las tildes generan problemas según como esté codificado.

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

Un problema habitual que perjudica gravemente la carga de datos es el empleo de caracteres especiales (tildes, ñ's). Una opción es realizar búsquedas y sustituciones mediante expresiones regulares.

Un problema muy habitual es que se utilicen distintas escalas de medida para medir una misma variable. Por ejemplo, una empresa multinacional puede declarar sus beneficios en España en euros y en Japón en yenes lo que provocaría un muy notorio desfase. Una solución para esto es realizar un pequeño análisis exploratorio de los datos una vez que estos han sido cargdados.

Una buena praxis para evitar datos duplicados es añadir a cada registro un número de identificación. En dicha variable se pueden eliminar los registros que aparecen duplicados.

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

Para iniciar el contexto spark basta con importar el módulo SparkContext:

In [1]:
from pyspark import SparkContext
sc = SparkContext()

Tras subir los datos a la imagen del Kitematic se procede a su lectura:

In [2]:
datos = "./partidosLigaNBA.csv"
raw_data = sc.textFile(datos)

En primer lugar se ojean los datos para ver cómo proceder al parseado:

In [3]:
raw_data.take(1)

['Date:Start..ET.:Visitor.Neutral:PTS:Home.Neutral:PTS.1']

Vemos que el archivo tiene una cabecera. Tomamos nota de ella y la eliminamos (desde un principio) porque nos genera problemas:

In [4]:
datos = "./sin_cabecera.csv"
raw_data = sc.textFile(datos)

Comprobamos que se ha resuelto el problema de la cabecera:

In [5]:
raw_data.take(10)

['Tue, Oct 30, 2007:"7:30 pm":Utah Jazz:117:Golden State Warriors:96',
 'Tue, Oct 30, 2007:"7:30 pm":Houston Rockets:95:Los Angeles Lakers:93',
 'Tue, Oct 30, 2007:"7:00 pm":Portland Trail Blazers:97:San Antonio Spurs:106',
 'Wed, Oct 31, 2007:"8:00 pm":Dallas Mavericks:92:Cleveland Cavaliers:74',
 'Wed, Oct 31, 2007:"8:30 pm":Seattle SuperSonics:103:Denver Nuggets:120',
 'Wed, Oct 31, 2007:"7:00 pm":Washington Wizards:110:Indiana Pacers:119',
 'Wed, Oct 31, 2007:"7:00 pm":San Antonio Spurs:104:Memphis Grizzlies:101',
 'Wed, Oct 31, 2007:"7:30 pm":Chicago Bulls:103:New Jersey Nets:112',
 'Wed, Oct 31, 2007:"7:00 pm":Sacramento Kings:90:New Orleans Hornets:104',
 'Wed, Oct 31, 2007:"7:00 pm":Milwaukee Bucks:83:Orlando Magic:102']

Antes de proceder al parseado se cuentan cuántos registros han sido cargado para asegurar que no se pierden registros al parsear:

In [6]:
raw_data.count()

12907

Se procede al parseado observando que los datos van separados por `":"`:

In [7]:
datos_tratados = raw_data.map(lambda x: x.split(":"))

Se comprueba que al parsear no se han perdido registros:

In [8]:
datos_tratados.count()

12907

El parseado se ha realizado de manera correcta.

Tomamos un elemento para ver cómo se ha parseado:

In [9]:
datos_tratados.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

Hay un pequeño problema con la hora. Como se ha partido por los `":"` tenemos que las horas y los minutos se separan en dos columnas. Tras revisar la práctica se observa que las horas no se van a considerar por lo que no nos moelestamos en arreglarlo pero es importante tenerlo en cuenta.

Así en cada línea tenemos la siguiente información:

- En la posición 0 la fecha.
- En la posición 1 la hora.
- En la posición 2 los minutos y si es am o pm.
- En la tercera el equipo local.
- En la cuarta el número de goles del equipo local.
- En la quinta el equipo visitante.
- En la sexta el número de goles del visitante.


Previo a responder a las preguntas del examen se procede a un breve análisis exploratorio del dataset.

### Análisis exploratorio

Se detecta que hay un problema. Para descubrir de qué se trata vemos qué valores distintos se presentan en la variables goles locales:

In [10]:
datos_tratados.map(lambda x: x[4]).distinct().collect()

['102',
 '128',
 '110',
 '88',
 '77',
 '66',
 '74',
 '73',
 '106',
 '56',
 '134',
 '57',
 '96',
 '132',
 '107',
 '64',
 '144',
 '70',
 '98',
 '68',
 '141',
 '93',
 '86',
 '63',
 '116',
 '82',
 '83',
 '112',
 '138',
 '119',
 '84',
 '140',
 '54',
 '121',
 '60',
 '122',
 '154',
 '139',
 '108',
 '143',
 'Playoffs',
 '113',
 '111',
 '91',
 '69',
 '115',
 '145',
 '100',
 '147',
 '130',
 '131',
 '127',
 '120',
 '62',
 '124',
 '104',
 '125',
 '135',
 '99',
 '151',
 '103',
 '126',
 '117',
 '95',
 '72',
 '118',
 '142',
 '80',
 '78',
 '109',
 '71',
 '114',
 '129',
 '92',
 '67',
 '123',
 '79',
 '101',
 '136',
 '133',
 '59',
 '75',
 '105',
 '89',
 '58',
 '61',
 '97',
 '94',
 '90',
 '65',
 '137',
 '85',
 '87',
 '81',
 '76']

De una ojeada se aprecia que en algunos casos hay una errata pues aparece `playoff`. Veamos cuántos casos como este se producen:

In [11]:
playoffs = datos_tratados.filter(lambda x: x[4]=="Playoffs")
playoffs.count()

10

Como son solo diez casos podemos examinarlos individualmente:

In [12]:
playoffs.take(10)

[['Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs'],
 ['Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs'],
 ['Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs'],
 ['Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs'],
 ['Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs'],
 ['Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs'],
 ['Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs'],
 ['Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs'],
 ['Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs'],
 ['Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs', 'Playoffs']]

Estas filas no producen información así que se decide eliminarlas.

In [13]:
datos_sin_playoffs = datos_tratados.filter(lambda x: x[4]!="Playoffs")

In [14]:
datos_sin_playoffs.count()

12897

Se recuerda que previamente había 12907 registros luego solo se han eliminado diez que se corresponden con los diez antes señalados.

In [15]:
stad_puntos_locales = datos_sin_playoffs.map(lambda x: int(x[4])).stats()
stad_puntos_locales

(count: 12897, mean: 98.73520973869917, stdev: 12.1262284681, max: 154.0, min: 54.0)

In [16]:
stad_puntos_visitantes = datos_sin_playoffs.map(lambda x: int(x[6])).stats()
stad_puntos_visitantes

(count: 12897, mean: 101.7780103900129, stdev: 12.2630490248, max: 168.0, min: 59.0)

Todos los registros se encuentran compleatados y los resultados se encuentran entre los parámetros razonables. El número mínimo de puntos es 54 y 59 respectivamente y el máximo son 154 y 168. Valores razonables para partidos en la NBA teniendoe en cuenta que son los valores más extremos.

Respecto al número de equipos participantes:

In [17]:
datos_sin_playoffs.map(lambda x: x[3]).distinct().count()

34

In [18]:
datos_sin_playoffs.map(lambda x: x[5]).distinct().count()

34

Hay el mismo número de equipos que juegan fuera y dentro de casa. Como de nuevo es un tamaño razonable se pueden ojear para comprobar que no hay problemas de transcripción de datos.

In [19]:
datos_sin_playoffs.map(lambda x: x[5]).distinct().collect()

['Oklahoma City Thunder',
 'Cleveland Cavaliers',
 'Minnesota Timberwolves',
 'Chicago Bulls',
 'Utah Jazz',
 'Memphis Grizzlies',
 'New Jersey Nets',
 'Miami Heat',
 'Brooklyn Nets',
 'Orlando Magic',
 'Indiana Pacers',
 'Houston Rockets',
 'San Antonio Spurs',
 'Seattle SuperSonics',
 'Washington Wizards',
 'Phoenix Suns',
 'Charlotte Hornets',
 'Sacramento Kings',
 'Milwaukee Bucks',
 'Dallas Mavericks',
 'New Orleans Hornets',
 'Philadelphia 76ers',
 'Denver Nuggets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Atlanta Hawks',
 'Toronto Raptors',
 'Boston Celtics',
 'Charlotte Bobcats',
 'Los Angeles Clippers',
 'Los Angeles Lakers',
 'New Orleans Pelicans',
 'New York Knicks',
 'Portland Trail Blazers']

Los equipos están bien transcritos.

Las fechas son un tema de vital importancia para los ejercicios así que se examinarán cuidadosamente:

En primer lugar se comprueba de qué años se poseén registros:

In [20]:
datos_sin_playoffs.map(lambda x: x[0][-4:]).distinct().collect()

['2014',
 '2013',
 '2017',
 '2016',
 '2011',
 '2012',
 '2007',
 '2015',
 '2010',
 '2008',
 '2009']

En las fechas se observa una tendencia:
- Si el día del mes poseé una cifra la fecha está formada por 16 caracteres, si no por 17.
- Las posiciones 5 a 7 (ambos inclusive) se corresponden con el mes.
- Las cuatro últimas posiciones se corresponden con el año.

In [21]:
datos_sin_playoffs.map(lambda x: x[0]).distinct().take(10)

['Sun, Mar 6, 2016',
 'Wed, Apr 27, 2016',
 'Thu, Jun 3, 2010',
 'Sun, Nov 17, 2013',
 'Sun, Jun 7, 2009',
 'Sat, Jan 22, 2011',
 'Tue, May 25, 2010',
 'Sun, Nov 18, 2012',
 'Sun, May 17, 2015',
 'Fri, Mar 25, 2016']

Hemos terminado de procesar los datos. Se procede a responder a las preguntas del examen:

In [22]:
datos = datos_sin_playoffs

## Quinta tarea: Media de la diferencia de puntos por año

En primer lugar se crea un RDD clave-valor siendo la clave el año en el que se jugó el partido el valor el valor absoluto de la diferencia de puntos entre los dos equipos:

In [23]:
diferencia_puntos_partidos = datos.map(lambda x: (x[0][-4:], abs(int(x[4]) - int(x[6]))))

Agrupando por claves obtenemos un diccionario en el que la clave es la misma que la del RDD y el valor una tupla con la suma de diferencias de puntos y el número de partidos:

In [26]:
year_difference_matches = diferencia_puntos_partidos.combineByKey(
    (lambda x: (x, 1)), # the initial value, with value x and count 1
    (lambda acc, value: (acc[0]+value, acc[1]+1)), # how to combine a pair value with the accumulator: sum value, and increment count
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # combine accumulators
)

year_difference_matches.collectAsMap()

{'2007': (5060, 456),
 '2008': (15376, 1332),
 '2009': (14595, 1316),
 '2010': (14358, 1321),
 '2011': (9435, 885),
 '2012': (15986, 1474),
 '2013': (14659, 1324),
 '2014': (14547, 1334),
 '2015': (14720, 1319),
 '2016': (15397, 1333),
 '2017': (9172, 803)}

Por último basta dividir dicha suma entre el número de partidos.

In [28]:
difference_average = year_difference_matches.map(lambda x: (x[0], round(x[1][0]/x[1][1],3))).collectAsMap()

El siguiente diccionario muestra un índice con el año y la media de diferencia de puntos:

In [29]:
difference_average

{'2007': 11.096,
 '2008': 11.544,
 '2009': 11.09,
 '2010': 10.869,
 '2011': 10.661,
 '2012': 10.845,
 '2013': 11.072,
 '2014': 10.905,
 '2015': 11.16,
 '2016': 11.551,
 '2017': 11.422}

## Sexta tarea: ¿Han judado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

In [30]:
partidos_en_casa = datos.map(lambda x: (x[3], x[0])) 

In [50]:
partidos_en_casa.countByKey()

defaultdict(int,
            {'Atlanta Hawks': 448,
             'Boston Celtics': 463,
             'Brooklyn Nets': 218,
             'Charlotte Bobcats': 283,
             'Charlotte Hornets': 127,
             'Chicago Bulls': 437,
             'Cleveland Cavaliers': 452,
             'Dallas Mavericks': 436,
             'Denver Nuggets': 424,
             'Detroit Pistons': 414,
             'Golden State Warriors': 440,
             'Houston Rockets': 434,
             'Indiana Pacers': 434,
             'Los Angeles Clippers': 430,
             'Los Angeles Lakers': 447,
             'Memphis Grizzlies': 434,
             'Miami Heat': 456,
             'Milwaukee Bucks': 414,
             'Minnesota Timberwolves': 402,
             'New Jersey Nets': 197,
             'New Orleans Hornets': 249,
             'New Orleans Pelicans': 166,
             'New York Knicks': 413,
             'Oklahoma City Thunder': 408,
             'Orlando Magic': 432,
             'Philadelphia 

In [51]:
partidos_fuera = datos.map(lambda x: (x[5], x[0]))

In [53]:
partidos_fuera.countByKey()

defaultdict(int,
            {'Atlanta Hawks': 448,
             'Boston Celtics': 467,
             'Brooklyn Nets': 217,
             'Charlotte Bobcats': 283,
             'Charlotte Hornets': 126,
             'Chicago Bulls': 436,
             'Cleveland Cavaliers': 449,
             'Dallas Mavericks': 431,
             'Denver Nuggets': 424,
             'Detroit Pistons': 415,
             'Golden State Warriors': 445,
             'Houston Rockets': 434,
             'Indiana Pacers': 434,
             'Los Angeles Clippers': 431,
             'Los Angeles Lakers': 450,
             'Memphis Grizzlies': 433,
             'Miami Heat': 461,
             'Milwaukee Bucks': 413,
             'Minnesota Timberwolves': 402,
             'New Jersey Nets': 197,
             'New Orleans Hornets': 250,
             'New Orleans Pelicans': 166,
             'New York Knicks': 412,
             'Oklahoma City Thunder': 410,
             'Orlando Magic': 431,
             'Philadelphia 

Se observan diferencias, por ejemplo, en los Boston Celtics. Esto se debe a que algunos equipos han cambiado de nombre a lo largo de la historia de la NBA.

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Clevelant?

En primer lugar nos quedamos solo con los partidos jugados en Enero:

In [31]:
partidos_Enero = datos.filter(lambda x: x[0][5:8] == 'Jan')

De ahí se extraen los partidos que el Cleveland juega en casa:

In [32]:
partidos_Enero_local = partidos_Enero.filter(lambda x: x[3] == 'Cleveland Cavaliers')

Y a continuación las victorias locales en Enero:

In [33]:
victoria_local_Clev_Enero = partidos_Enero_local.filter(lambda x: int(x[4]) > int(x[6]))

In [34]:
victoria_local_Clev_Enero.count()

41

Análogamente para los partidos jugado como visitante:

In [35]:
partidos_Enero_visit = partidos_Enero.filter(lambda x: x[5] == 'Cleveland Cavaliers')

In [36]:
victoria_visit_Clev_Enero = partidos_Enero_visit.filter(lambda x: int(x[4]) < int(x[6]))

In [37]:
victoria_visit_Clev_Enero.count()

42

In [38]:
print("El Cleveland Cavaliers ha ganado en Enero ", format(victoria_visit_Clev_Enero.count() + victoria_local_Clev_Enero.count()), "partidos.")

El Cleveland Cavaliers ha ganado en Enero  83 partidos.


## Octava pregunta: ¿Los Warriors son mejores fuera de casa o en casa?

Para calcular el número de victorias locales de los Warriors calculamos en primer lugar el número de partidos que juegan en casa:

In [39]:
partidos_locales_Warriors = datos.filter(lambda x: x[3] == 'Golden State Warriors' )

Tras ello se calcula el número de victorias en dichos partidos:

In [40]:
partidos_locales_ganados_Warriors = partidos_locales_Warriors.filter(lambda x: int(x[4]) > int(x[6]))

El número de victorias locales de los Warriors es:

In [41]:
partidos_locales_ganados_Warriors.count()

215

Análogamente para los partidos jugados como visitante:

In [42]:
partidos_visitante_Warriors = datos.filter(lambda x: x[5] == 'Golden State Warriors' )
partidos_visitante_ganados_Warriors = partidos_visitante_Warriors.filter(lambda x: int(x[4]) < int(x[6]))
partidos_visitante_ganados_Warriors.count()

308

__Solución:__ Los Warriors son mejores fuera de casa con 308 victorias fuera frente a 215 victorias locales.

## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

Lo primero es ver qué equipos ganan en casa:

In [43]:
victorias_locales = datos.filter(lambda x: int(x[4]) > int(x[6]))

Se crea un RDD que posee el año y el nombre del equipo ganador:

In [44]:
victorias_locales1 = victorias_locales.map(lambda x: (x[0][-4:], x[3])) 

In [45]:
victorias_locales1.take(5)

[('2007', 'Utah Jazz'),
 ('2007', 'Houston Rockets'),
 ('2007', 'Dallas Mavericks'),
 ('2007', 'San Antonio Spurs'),
 ('2007', 'Detroit Pistons')]

Análogamente para las victorias fuera de casa:

In [46]:
victorias_visitantes = datos.filter(lambda x: int(x[4]) < int(x[6]))

In [47]:
victorias_visitantes1 = victorias_visitantes.map(lambda x: (x[0][-4:], x[5])) 

Uno ambos conjuntos :

In [48]:
victorias = (victorias_locales1 + victorias_visitantes1)

In [49]:
victorias.take(10)

[('2007', 'Utah Jazz'),
 ('2007', 'Houston Rockets'),
 ('2007', 'Dallas Mavericks'),
 ('2007', 'San Antonio Spurs'),
 ('2007', 'Detroit Pistons'),
 ('2007', 'Phoenix Suns'),
 ('2007', 'Houston Rockets'),
 ('2007', 'Philadelphia 76ers'),
 ('2007', 'Denver Nuggets'),
 ('2007', 'Toronto Raptors')]

Seleccionar el primero de cada temporada. Contar para cada equipo el número de temporadas. Ordenar

In [50]:
datos.map(lambda x: x[0][-4:]).distinct().count()

11

In [51]:
victorias3 = victorias.map(lambda x: ((x[0],x[1]), 1))

In [52]:
victorias3.take(10)

[(('2007', 'Utah Jazz'), 1),
 (('2007', 'Houston Rockets'), 1),
 (('2007', 'Dallas Mavericks'), 1),
 (('2007', 'San Antonio Spurs'), 1),
 (('2007', 'Detroit Pistons'), 1),
 (('2007', 'Phoenix Suns'), 1),
 (('2007', 'Houston Rockets'), 1),
 (('2007', 'Philadelphia 76ers'), 1),
 (('2007', 'Denver Nuggets'), 1),
 (('2007', 'Toronto Raptors'), 1)]

In [53]:
temp_team_victorias = victorias3.map(lambda line: (line[0],1)).reduceByKey(lambda a, b: a + b)

In [54]:
temp_team_victorias = temp_team_victorias.map(lambda x: (x[0][0],(x[0][1],x[1])))

In [55]:
temp_team_victorias.sortByKey().sortBy(lambda x: (-x[1][1])).takeOrdered(300, lambda x: -x[1][1])

[('2015', ('Golden State Warriors', 88)),
 ('2016', ('Golden State Warriors', 87)),
 ('2013', ('Miami Heat', 85)),
 ('2008', ('Boston Celtics', 84)),
 ('2012', ('San Antonio Spurs', 82)),
 ('2009', ('Los Angeles Lakers', 81)),
 ('2012', ('Oklahoma City Thunder', 79)),
 ('2012', ('Miami Heat', 79)),
 ('2008', ('Los Angeles Lakers', 77)),
 ('2016', ('Cleveland Cavaliers', 77)),
 ('2009', ('Cleveland Cavaliers', 76)),
 ('2013', ('San Antonio Spurs', 73)),
 ('2014', ('San Antonio Spurs', 73)),
 ('2016', ('San Antonio Spurs', 72)),
 ('2010', ('Los Angeles Lakers', 71)),
 ('2009', ('Orlando Magic', 70)),
 ('2015', ('Cleveland Cavaliers', 70)),
 ('2012', ('Los Angeles Clippers', 68)),
 ('2016', ('Toronto Raptors', 68)),
 ('2010', ('Orlando Magic', 67)),
 ('2013', ('Indiana Pacers', 67)),
 ('2010', ('Boston Celtics', 66)),
 ('2013', ('Oklahoma City Thunder', 66)),
 ('2015', ('San Antonio Spurs', 66)),
 ('2015', ('Atlanta Hawks', 66)),
 ('2012', ('Chicago Bulls', 65)),
 ('2008', ('Cleveland Cav

__Solución.__ El Golden State Warriors es el mejor en 2015, 2016 y 2017, en 2013 Miami Heat, en 2008 los Boston Celtics, en 2012 y 2014 San Antonio Spurs, en 2009 y 2010 los Angeles Lakers, en 2011 Chicago Bulls y en 2007 Boston Celtics.

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com

Cargamos el texto en una variable:

In [56]:
texto = 'Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com'

Importamos el módulo de expresiones regulares:

In [57]:
import re

El patrón del teléfono son tres cifras, un espacio, dos cifras, un espacio, dos cifras, un espacio y dos cifras:

In [58]:
patron_telefono = re.compile('\d{3}\s\d{2}\s\d{2}\s\d{2}')

In [59]:
patron_telefono.findall(texto)

['902 13 23 13', '902 88 30 08', '902 88 88 35']

Para la expresión regular del mail elegimos palabras entre 3 y 15 caracteres (letras o números) seguidos de @bankinter.com. 

In [60]:
patron_mail = re.compile('\w{3,15}@bankinter.com')

In [61]:
patron_mail.findall(texto)

['atencioncliente@bankinter.com']